In [108]:
import requests
import pandas as pd
from shapely.geometry import Point
import geopandas as gpd 
import argparse

In [129]:
def embajadas():
    response = requests.get('https://datos.madrid.es/egob/catalogo/201000-0-embajadas-consulados.json')
    results = response.json()
    embajadas = pd.json_normalize(results['@graph'])
    embajadas["Type of place"] = "Embajadas y consulados"
    embajadas["Dirección"] = embajadas["address.street-address"]
    embajadas["location.latitude"] = pd.to_numeric(embajadas["location.latitude"],errors = 'coerce')
    embajadas["location.longitude"] = pd.to_numeric(embajadas["location.longitude"],errors = 'coerce')
    embajadas_clean = embajadas[["title","Type of place", "Dirección","location.longitude","location.latitude"]]
    embajadas_clean = embajadas_clean.rename(columns={"title": "Place of interest", "Dirección": "Place address", "location.longitude": "Longitud_inicial", "location.latitude": "Latitud_inicial"}, errors="raise")
    return embajadas_clean

def estaciones():
    estaciones = pd.read_json("../data/bicimad.json", orient='records')
    geometry_coordinates = estaciones["geometry_coordinates"].str.split(expand=True)
    geometry_coordinates.columns = ['LONGITUD', 'LATITUD']
    geometry_coordinates['LONGITUD'] = geometry_coordinates['LONGITUD'].str.replace("[","")
    geometry_coordinates['LONGITUD'] = geometry_coordinates['LONGITUD'].str.replace(",","")
    geometry_coordinates['LATITUD'] = geometry_coordinates['LATITUD'].str.replace("]","")
    estaciones = pd.concat([estaciones, geometry_coordinates], axis=1)
    estaciones_clean = estaciones.rename(columns={"name": "BiciMAD station", "address": "Station location", "LONGITUD": "Longitud_final", "LATITUD": "Latitud_final"}, errors="raise")
    estaciones_clean = estaciones_clean[["BiciMAD station","Station location","Longitud_final","Latitud_final"]]
    estaciones_clean['Latitud_final'] = pd.to_numeric(estaciones_clean['Latitud_final'],errors = 'coerce')
    estaciones_clean['Longitud_final'] = pd.to_numeric(estaciones_clean['Longitud_final'],errors = 'coerce')
    return estaciones_clean


In [131]:
def to_mercator(lat, long):
    # transform latitude/longitude data in degrees to pseudo-mercator coordinates in metres
    c = gpd.GeoSeries([Point(lat, long)], crs=4326)
    c = c.to_crs(3857)
    return c
def distance_meters(mercator_start, mercator_finish):
    # return the distance in metres between to latitude/longitude pair point in degrees (i.e.: 40.392436 / -3.6994487)
    return mercator_start.distance(mercator_finish)


In [132]:
#embajadas = pd.read_json('../data/embajadas.json')
#estaciones = pd.read_json('../data/estaciones.json')

In [133]:

def mercator_emb():
    embajadas_clean["mercator_start"] = embajadas_clean.apply(lambda x: to_mercator(x['Latitud_inicial'],x['Longitud_inicial']),axis=1)
    return embajadas_clean
def mercator_est():
    stations_clean["mercator_finish"] = estaciones_clean.apply(lambda x: to_mercator(x['Latitud_final'],x['Longitud_final']),axis=1)
    return estaciones_clean


In [138]:
def merge():
    df_resultado = pd.merge(embajadas_clean,estaciones_clean, how="cross")
    return df_resultado

In [145]:
def apply_distance():
    df_resultado["Distance"] = df_resultado.apply(lambda x: distance_meters(x['mercator_start'],x['mercator_finish']),axis=1)
    return df_resultado

In [146]:
def all_places():  
    return df_resultado.sort_values(by = "Distance", ascending = True).groupby('Place of interest')['Type of place','Place address','BiciMAD station', 'Station location','Distance'].nth(0).drop(["Distance"], axis = "columns") 

In [147]:
all_places()

KeyError: 'Distance'

In [74]:
#estaciones_clean = estaciones()
#embajadas_clean = embajadas()
#embajadas_clean = mercator_emb()
#estaciones_clean = mercator_est()
#df_resultado = merge()
#df_resultado = apply_distancia()
#all_minimum()

In [61]:
def minimum():
    df_minimo = df_resultado[df_resultado["Place of interest"] == input('Pon el lugar de interés: ')]
    df_minimum = df_minimo[df_minimo["Distance"] == df_minimo["Distance"].min()]
    minimum_mod = df_minimum[["Place of interest","Type of place", "Place address","BiciMAD station","Station location"]]
    return minimum_mod

In [62]:
minimum()

Pon el lugar de interés: Embajada de Yemen


,Place of interest,Type of place,Place address,BiciMAD station,Station location
41935,Embajada de Yemen,Embajadas y consulados,PASEO CASTELLANA 117 PLANTA 8 D,Pavones,Calle Hacienda de Pavones nº 342
